This is the Jupyter notebook for team F. We are using a comprehensive synthetic dataset of 1500 patients diagnosed with Obsessive Compulsive Disorder (OCD). This dataset is available on Kaggle.  
  
Reference:   
Haque O, Alamgir Z. OCD Patient Dataset: Demographics and Clinical Data [Internet]. 2023 [cited 2024-09-08]. Available from: https://www.kaggle.com/datasets/ohinhaque/ocd-patient-dataset-demographics-and-clinical-data/data

The dataset looks as follows:

In [ ]:
pip install pandas

In [ ]:
# read dataset
import pandas as pd
df = pd.read_csv("ocd_patient_dataset.csv")
df.head()

The dataset includes demographic information such as patient id, age, gender, ethnicity, marital status and education level; along with clinical details like OCD diagnosis date, symptom duration, past history of psychiatric diagnosis, and family history of OCD. Moreover, symptom severity is assessed through the Yale-Brown Obsessive Compulsive Scale (Y-BOCS) for obsessive and compulsive symptom types. Furthermore, there is record of other mental health conditions such as anxiety and depression, and the medications prescribed to the patients.  
  
Reference:  
Goodman WK, Lawrence HP, Rasmussen SA, Mazure C, Fleischmann RL, Hill CL, et al. The Yale-Brown Obsessive Compulsive Scale. Arch Gen Psychiatry. 1989;46:1006-11

Over the next few sections, the following tasks have been carried out:  
1. The variable datatypes have been checked  
2. The variables that had "yes" or "no answers were converted to Boolean  
3. The missing values have been taken care of.  


In [ ]:
# display variable datatypes
df.dtypes

In [ ]:
# convert to boolean
df = pd.read_csv("ocd_patient_dataset.csv", true_values =["Yes"], false_values=["No"])
df.head()
df.dtypes

In [ ]:
# missing values
df.isnull().sum()

In [ ]:
# 'Previous Diagnoses' and 'Medications' have missing values, however we have assumed that it implies that the patient does not have a diagnosis or is not taking any medication respectively.
# fill missing values with the word "None"
df['Previous Diagnoses'] = df['Previous Diagnoses'].fillna('None')
df['Medications'] = df['Medications'].fillna('None')
df.isnull().sum()

The Y-BOCS scale is a standardized clinical assessment tool designed to evaluate the severity and type of symptoms in individuals with OCD. There are 10 items: 5 for grading Obsession and 5 for grading compulsion and each can be scored from 0-4. Thus, scores for obsession and compulsion range from 0-20 each, and the total score ranges from 0-40. The dataset did not have the total score calculated which we have done below by adding the scores for Y-BOCS Score (Obsessions) and Y-BOCS Score (Compulsions) and stored the values under the feature name "Total_Score".

In [7]:
# calculate total score
df['Total_Score'] = df['Y-BOCS Score (Obsessions)'] + df['Y-BOCS Score (Compulsions)']

We realised that there were instances in our dataset that had total score above 40, and individual scores for Obsessions and Compulsions above 20 each, which is not acceptable based on the scoring system of the scale. Hence, we have eliminated those.

In [ ]:
# count above 40 for Total_Score
count_above_40 = (df['Total_Score'] > 40).sum()
print(f'Number of values above 40 for Total_Score: {count_above_40}')

# count above 20 for Y-BOCS Score (Obsessions)
count_above_20_O = (df['Y-BOCS Score (Obsessions)'] > 20).sum()
print(f'Number of values above 20 for Obsession score: {count_above_20_O}')

# count above 20 for Y-BOCS Score (Compulsions)
count_above_20_C = (df['Y-BOCS Score (Compulsions)'] > 20).sum()
print(f'Number of values above 20 for Compulsion score: {count_above_20_C}')



In [ ]:
# create a dataframe with valid scores
filtered_df = df[
    (df['Y-BOCS Score (Obsessions)'] <= 20) &
    (df['Y-BOCS Score (Compulsions)'] <= 20) &
    (df['Total_Score'] <= 40)
]
filtered_df

Saving the filtered dataframe in a csv

In [10]:
filtered_df.to_csv('filtered_df.csv', index=False)

We were interested in predicting the severity of OCD given by "Total_Score", and based on a literature review found that age, duration of symptoms, family history of OCD, and diagnosis of Anxiety or Depression affect it. Gender being a basic demographic feature has been retained though research does not clearly indicate its influence on OCD. Obsession Type and Compulsion Type are a reflection of patient's symptoms- symptoms are one of the earliest and vital information collected and we believe may influence severity of the disorder, and therefore have been retained.  

References:  
1. Mathes BM, Morabito DM, Schmidt NB. Epidemiological and Clinical Gender Differences in OCD. Curr Psychiatry Rep. 2019 Apr 23;21(5):36. doi: 10.1007/s11920-019-1015-2.
2. Riddle DB, Guzick A, Minhajuddin A, Smárason O, Armstrong GM, Slater H, et al. Obsessive-compulsive disorder in youth and young adults with depression: Clinical characteristics of comorbid presentations. J Obsessive Compuls Relat Disord. 2023 Jul;38:100820. doi: 10.1016/j.jocrd.2023.100820. 
3. Zheng H, Zhang Z, Huang C, Luo G. Medical status of outpatients with obsessive-compulsive disorder in psychiatric department and its influencing factors. Zhong Nan Da Xue Xue Bao Yi Xue Ban. 2022 Oct 28;47(10):1418-1424. English, Chinese. doi: 10.11817/j.issn.1672-7347.2022.220125. 
4. Mahjani B, Bey K, Boberg J, Burton C. Genetics of obsessive-compulsive disorder. Psychol Med. 2021 Oct;51(13):2247-2259. doi: 10.1017/S0033291721001744. Epub 2021 May 25. PMID: 34030745; PMCID: PMC8477226.
5. Mathes BM, Morabito DM, Schmidt NB. Epidemiological and Clinical Gender Differences in OCD. Curr Psychiatry Rep. 2019 Apr 23;21(5):36. doi: 10.1007/s11920-019-1015-2. PMID: 31016410.

With the goal of predicting OCD severity, it was necessary to label the Total_Score for classification task. We tried to  identify the distribution of scores across bins of 5 from 0-40 in order to define score categories.

In [ ]:
# Define bin ranges (0 to 40 with bins of size 5)
bins = range(0, 45, 5)  # This creates bins from 0-5, 6-10, ..., 36-40

# Use pd.cut to categorize the Total_Score into bins
filtered_df['Score_Bins'] = pd.cut(df['Total_Score'], bins=bins, right=False)

# Calculate the count for each bin
bin_counts = filtered_df['Score_Bins'].value_counts().sort_index()

# Display the bin counts
print(bin_counts)

Based on the distribution above, we have decided to categorise Total_Score as follows:  
0-20: Low
21-40: High  
We have stored it as "Score_Category"

In [ ]:
# Categorize Total_Score into 'Low' and 'High' based on the given range
filtered_df['Score_Category'] = pd.cut(filtered_df['Total_Score'], 
                              bins=[0, 20, 40], 
                              labels=['Low', 'High'], 
                              include_lowest=True)

#  View a concise dataframe based on the feature selection
filtered_df = filtered_df[["Age", "Gender", "Family History of OCD", "Duration of Symptoms (months)", "Obsession Type", "Compulsion Type", "Total_Score", "Depression Diagnosis", "Anxiety Diagnosis", "Score_Category"]]
filtered_df.head()

### Descriptive Analytics  
In this section, we have tried to analyse if the literature review results hold true for our dataset.

In [13]:
import plotly.express as px
import plotly.graph_objects as go

We tried to evaluate if our data is equally distributed for age and gender which are important demographic features and Score_Category which will be the task.  
The age has been split into age groups and we have tried to evaluate how our dataset is represented across the various age bins.

In [ ]:
# Calculate Average Age
average_age = filtered_df['Age'].mean()
print(f"Average Age: {average_age}")

# Create age groups (bins)
filtered_df['Age Group'] = pd.cut(filtered_df['Age'], bins=[0, 18, 30, 50, 70, 100], 
                         labels=['0-18', '19-30', '31-50', '51-70', '71+'])

# Count the number of males and females in each age group
gender_age_group = filtered_df.groupby(['Age Group', 'Gender']).size().unstack().fillna(0)

# Add totals for each bin (row-wise) and overall totals (column-wise)
gender_age_group['Total'] = gender_age_group.sum(axis=1)  # Total people in each bin
gender_age_group.loc['Total'] = gender_age_group.sum()     # Total males and females across all bins

# Display the table
print(gender_age_group)


# Create a figure
fig0 = go.Figure()

# Add bar plot for males
fig0.add_trace(
    go.Bar(
        x=gender_age_group.index,
        y=gender_age_group['Male'],
        name='Male',
        marker_color='blue'
    )
)

# Add bar plot for females
fig0.add_trace(
    go.Bar(
        x=gender_age_group.index,
        y=gender_age_group['Female'],
        name='Female',
        marker_color='pink'
    )
)

# Update layout to create stacked bars
fig0.update_layout(
    barmode='stack',  # Stacked bar mode
    title='Number of People by Age Group and Gender',
    xaxis_title='Age Group',
    yaxis_title='Number of People',
    legend_title='Gender',
    xaxis_tickangle=0,
    yaxis=dict(showgrid=True)
)

# Show the plot
fig0.show()

The above results indicate that across most age group bins (except 0-18 and 31-50), females are more in number than males, and therefore total number of females is more than males. However the imbalance does not seem significant.  
The representation across age group bins is as expected with most instances falling in the adults to middle age groups.

In [ ]:
# Count the instances for each Score_Category
score_category_counts = filtered_df['Score_Category'].value_counts()

# Create a bar plot using Plotly to visualize the counts of 'Low' and 'High'
fig00 = px.bar(x=score_category_counts.index, 
             y=score_category_counts.values, 
             labels={'x': 'Score Category', 'y': 'Count of Instances'}, 
             title='Count of Instances in Low and High Score Categories',
             text=score_category_counts.values)  # Display counts on bars

# Customize the layout
fig00.update_layout(xaxis_title='Score Category', 
                  yaxis_title='Number of Instances',
                  showlegend=False, bargap=0.8)

# Show the figure
fig00.show()

Both score categories seem to be represented satisfactorily without significant imbalance.

1. What is the distribution of OCD severity (Total_Score) across different age groups?  
This question helps to determine if certain age groups tend to experience more severe OCD symptoms. It might reveal whether younger or older people are more affected.

In [ ]:
# Box plot to show distribution
fig = px.box(filtered_df, x='Age Group', y='Total_Score', title='Distribution of OCD Severity Across Age Groups',
             labels={'Age Group': 'Age Group', 'Total_Score': 'OCD Severity (Y-BOCS Total Score)'},
             category_orders={'Age Group': ['0-18', '19-30', '31-50', '51-70', '71+']})  # Ensure proper order

fig.show()

All age groups have a similar distribution, and the total scores mostly lie between over 10 and below 30. Only age groups over 19 reach up to nearly 40 (maximum value)

2. Is there a significant difference in OCD severity based on gender?  
Literature does not clearly indicate a relation between gender and OCD. Gender-based analysis can uncover any significant differences between males and females in terms of OCD severity. This could help in targeting gender-specific interventions.

In [ ]:
# Box plot for OCD severity (Total_Score) by Gender
fig3 = px.box(filtered_df, x='Gender', y='Total_Score', title='Distribution of OCD Severity by Gender',
              labels={'Total_Score': 'OCD Severity (Y-BOCS Total Score)', 'Gender': 'Gender'})
fig3.show()

It is seen that females have a distribution of higher total scores than males. The total scores of most females range from 15 - 29, but those of males have only a lower range from 12 -24.

3. How does family history of OCD influence the Total_Score?  
We try to examine whether individuals with a family history of OCD are more likely to have severe symptoms, potentially highlighting genetic or environmental influences.  
The left y axis represents number of people while the y axis on the right indicates the Total_Score. The bar plot represents number of instances belonging to classess 'True' or 'False' for Family history of OCD.

In [18]:
import plotly.io as pio

In [ ]:
# Count the number of people with and without family history of OCD
family_history_counts = filtered_df['Family History of OCD'].value_counts()

# Create a bar plot for the number of people with and without family history of OCD
bar_trace = go.Bar(
    x=family_history_counts.index, 
    y=family_history_counts.values, 
    name='Number of People',
    yaxis='y1',
    marker_color='lightblue'
)

# Create a box plot for the distribution of Total_Score for each category
box_trace = go.Box(
    x=filtered_df['Family History of OCD'], 
    y=filtered_df['Total_Score'], 
    name='Distribution of Total Score',
    yaxis='y2',
    marker_color='orange'
)

# Combine the bar and box plot using secondary y-axes
fig4 = go.Figure(data=[bar_trace, box_trace])

# Update layout for dual y-axes
fig4.update_layout(
    title='Number of People with Family History of OCD and Distribution of Their Scores',
    xaxis_title='Family History of OCD',
    yaxis=dict(
        title='Number of People',
        showgrid=False
    ),
    yaxis2=dict(
        title='Distribution of Total Score',
        overlaying='y',  # Overlay on the same plot
        side='right'
    ),
    legend=dict(x=0.1, y=1.1)
)

# Show the figure
fig4.show()

It shows that most patients without family history have a higher score range (from around 10 to nearly 30) while those having family history have a range up to 25, which is in fact somewhat contradictory to literature which shows that a positive family history leads to more severe OCD.

4. How does diagnosis of Anxiety or Depression affect Total_Score?  
We try to examine whether having a diagnosis of Anxiety or Depression affects the severity of OCD.  
The right y axis represents number of people while the y axis on the left indicates the Total_Score.  
The patients have ben divided into 4 cohorts: those diagnosed with anxiety and depression, those diagnosed with only anxiety, those only diagnosed with depression and those without anxiety or depression. The bar plot shows number of instances belonging to each cohort.

In [ ]:
# Add a 'Condition' column for easy visualization
filtered_df['Condition'] = filtered_df.apply(lambda row: 
                            'Anxiety T, Depression T' if row['Anxiety Diagnosis'] and row['Depression Diagnosis'] else
                            'Anxiety T, Depression F' if row['Anxiety Diagnosis'] and not row['Depression Diagnosis'] else
                            'Anxiety F, Depression T' if not row['Anxiety Diagnosis'] and row['Depression Diagnosis'] else
                            'Anxiety F, Depression F', axis=1)

# Count the number of people in each condition group
condition_counts = filtered_df['Condition'].value_counts().reset_index()
condition_counts.columns = ['Condition', 'Count']

# Create a bar plot to show the number of people in each group
bar_trace = go.Bar(
    x=condition_counts['Condition'], 
    y=condition_counts['Count'], 
    text=condition_counts['Count'],  # Show count labels on bars
    textposition='auto',
    name='Count of People',  # Legend label for bar chart
    marker_color='turquoise',
    opacity=0.6,
    yaxis='y2'  # Map bar plot to a secondary y-axis
)

# Create a boxplot to visualize the total score distribution across different conditions using Plotly
fig6 = px.box(filtered_df, 
             x='Condition', 
             y='Total_Score', 
             title='Total Score Distribution by Anxiety and Depression Conditions',
             labels={'Condition': 'Anxiety and Depression Conditions', 'Total_Score': 'Total Score'},
             category_orders={'Condition': ['Anxiety T, Depression T', 'Anxiety T, Depression F', 
                                            'Anxiety F, Depression T', 'Anxiety F, Depression F']}
            )

# Add bar plot trace to the figure
fig6.add_trace(bar_trace)

# Update layout to combine box and bar plots
fig6.update_layout(
    xaxis_title='Condition',
    yaxis_title='Total Score',
    xaxis_tickangle=45,
    yaxis=dict(showgrid=True, title='Total Score'),  # Primary y-axis for Total Score
    yaxis2=dict(title='Number of People', overlaying='y', side='right'),  # Secondary y-axis for count of people
    legend=dict(x=1, y=1),  # Position the legend
    title='Total Score Distribution and Count of People by Anxiety and Depression Conditions'
)
# Show the plot
fig6.show()

Patients having only one condition: anxiety or depression, have a higher score range from over 10 to nearly 30, while those having both conditions have a lowest range of up to 24. Those without both conditions can have a slightly higher range up to 26.

5. Are the Obsession types represented equally in this dataset? What is the average Total_Score for each type?  
The Y-BOCS Scale identifies 5 types of Obsessions: Contamination, Harm-related, Hoarding, Symmetry and Religious. Based on the majority of patient's symptoms and the patterns in their behaviour, they are diagnosed to be having one of these types. The left y axis shows the number of instances belonging to each obsession type and is denoted by the bar plot. The y axis on the right represents Total_Score and the yellow line plot marks the average total Y-BOCS score for each obsession type.

In [ ]:
# Count the number of instances for each obsession type
obsession_counts = filtered_df['Obsession Type'].value_counts()

# Calculate the average Total Score for each obsession type
obsession_avg_scores = filtered_df.groupby('Obsession Type')['Total_Score'].mean()

# Create a figure
fig7 = go.Figure()

# Add bar plot for the number of instances (left y-axis)
fig7.add_trace(
    go.Bar(
        x=obsession_counts.index,
        y=obsession_counts.values,
        name='Number of Instances',
        marker_color='lightblue',
        text=obsession_counts.values,  # Show count on the bars
        textposition='auto'
    )
)

# Add line plot for the average total score (right y-axis)
fig7.add_trace(
    go.Scatter(
        x=obsession_avg_scores.index,
        y=obsession_avg_scores.values,
        name='Average Total Score',
        yaxis='y2',  # Associate this with the second y-axis
        mode='lines+markers',
        line=dict(color='orange', width=3),
        marker=dict(size=8),
        text=obsession_avg_scores.values,  # Show average scores on the points
        textposition='top center'
    )
)

# Update layout to include a second y-axis
fig7.update_layout(
    title='Number of Instances and Average Total Score by Obsession Type',
    xaxis_title='Obsession Type',
    yaxis_title='Number of Instances',
    yaxis2=dict(
        title='Average Total Score',
        overlaying='y',  # Overlay y-axis 2 on the same plot as y-axis 1
        side='right'
    ),
    legend=dict(x=0.1, y=1.1),
    xaxis_tickangle=45,
    yaxis=dict(showgrid=True)
)

# Show the plot
fig7.show()

Among the 5 types, contamination and harm-related have the highest number of instances over 88, while hoarding and symmetry have a slightly lower number, around 83 but the religious type has only 77. Regarding average total score, the harm-related type has the highest (20.7), and contamination and hoarding, around 20 while symmetry and religious types have the lowest score, just above 19.

6. Are the Compulsion types represented equally in this dataset? What is the average Total_Score for each type?  
The Y-BOCS Scale identifies 5 types of Compulsions: Checking, Washing, Ordering, Counting, Praying. Based on the majority of patient's symptoms and the patterns in their behaviour, they are diagnosed to be having one of these types. The left y axis shows the number of instances belonging to each compulsion type and is denoted by the bar plot. The y axis on the right represents Total_Score and the yellow line plot marks the average total Y-BOCS score for each compulsion type.

In [ ]:
# Count the number of instances for each compulsion type
compulsion_counts = filtered_df['Compulsion Type'].value_counts()

# Calculate the average Total Score for each compulsion type
compulsion_avg_scores = filtered_df.groupby('Compulsion Type')['Total_Score'].mean()

# Create a figure
fig8 = go.Figure()

# Add bar plot for the number of instances (left y-axis)
fig8.add_trace(
    go.Bar(
        x=compulsion_counts.index,
        y=compulsion_counts.values,
        name='Number of Instances',
        marker_color='lightblue',
        text=compulsion_counts.values,  # Show count on the bars
        textposition='auto'
    )
)

# Add line plot for the average total score (right y-axis)
fig8.add_trace(
    go.Scatter(
        x=compulsion_avg_scores.index,
        y=compulsion_avg_scores.values,
        name='Average Total Score',
        yaxis='y2',  # Associate this with the second y-axis
        mode='lines+markers',
        line=dict(color='orange', width=3),
        marker=dict(size=8),
        text=compulsion_avg_scores.values,  # Show average scores on the points
        textposition='top center'
    )
)

# Update layout to include a second y-axis
fig8.update_layout(
    title='Number of Instances and Average Total Score by Compulsion Type',
    xaxis_title='Compulsion Type',
    yaxis_title='Number of Instances',
    yaxis2=dict(
        title='Average Total Score',
        overlaying='y',  # Overlay y-axis 2 on the same plot as y-axis 1
        side='right'
    ),
    legend=dict(x=0.1, y=1.1),
    xaxis_tickangle=45,
    yaxis=dict(showgrid=True)
)

# Show the plot
fig8.show()

Checking and washing have the highest number of instances, about 90, counting has a lower number, 83 while ordering and praying account for only just above 78. However, praying has the highest average total score over 22. Checking and ordering have lower scores just above 20. Counting accounts for the score, about 19 and washing has the lowest, over 17.

To prepare the dataframe for diagnosis and prediction, over the next few codes, the following has been done:
1. Conversion of boolean values to integers
2. Normalisation of numerical features using Min-Max scaler
3. Conversion of categorical features to numerical by one hot encoding
4. Using Label encoding on Score_Category

In [ ]:
# Convert boolean values to integers (0 for False, 1 for True) for Family history of OCD
filtered_df['Family History of OCD'] = filtered_df['Family History of OCD'].astype(int)

# Convert boolean values to integers (0 for False, 1 for True) for Depression Diagnosis
filtered_df['Depression Diagnosis'] = filtered_df['Depression Diagnosis'].astype(int)

# Convert boolean values to integers (0 for False, 1 for True) for Anxiety Diagnosis
filtered_df['Anxiety Diagnosis'] = filtered_df['Anxiety Diagnosis'].astype(int)

# Convert Gender to Boolean: Male as True, Female as False
filtered_df['Gender_Boolean'] = filtered_df['Gender'].map({'Male': True, 'Female': False})

# Convert Boolean to Integers (True -> 1, False -> 0)
filtered_df['Gender_Boolean'] = filtered_df['Gender_Boolean'].astype(int)

In [ ]:
# Normalisation of numerical features
from sklearn.preprocessing import MinMaxScaler

# Initialize Min-Max Scaler
scaler = MinMaxScaler(clip=True)

# Select the columns you want to normalize
columns_to_normalize = ['Age', 'Duration of Symptoms (months)']

#print(filtered_df[columns_to_normalize].describe())

# Apply the Min-Max scaling
filtered_df[columns_to_normalize] = scaler.fit_transform(filtered_df[columns_to_normalize])

#print(scaler.data_min_,)

import joblib

# Save the scaler to a file
joblib.dump(scaler, 'scaler.pkl')

In [25]:
# One hot encoding for Obsession Type and Compulsion Type

from sklearn.preprocessing import OneHotEncoder

# Initialize OneHotEncoder with sparse_output=False to return dense array
encoder = OneHotEncoder(sparse_output=False)

# Apply One-Hot Encoding to the 'Obsession Type' column
one_hot_encoded = encoder.fit_transform(filtered_df[["Obsession Type"]])

# Get the feature names for the encoded columns
encoded_feature_names = encoder.get_feature_names_out(["Obsession Type"])

# Convert the encoded data into a DataFrame
df_one_hot_O = pd.DataFrame(one_hot_encoded, columns=encoded_feature_names)

# Reset the index to avoid row misalignment during concatenation
df_one_hot_O.reset_index(drop=True, inplace=True)

# Concatenate the original dataframe with the new one-hot encoded columns
filtered_df.reset_index(drop=True, inplace=True)
filtered_df = pd.concat([filtered_df, df_one_hot_O], axis=1)

# Drop the original 'Obsession Type' column if you no longer need it
filtered_df.drop(columns=['Obsession Type'], inplace=True)

# Apply One-Hot Encoding to the 'Compulsion Type' column
one_hot_encoded = encoder.fit_transform(filtered_df[["Compulsion Type"]])

# Get the feature names for the encoded columns
encoded_feature_names = encoder.get_feature_names_out(["Compulsion Type"])

# Convert the encoded data into a DataFrame
df_one_hot_C = pd.DataFrame(one_hot_encoded, columns=encoded_feature_names)

# Reset the index to avoid row misalignment during concatenation
df_one_hot_C.reset_index(drop=True, inplace=True)

# Concatenate the original dataframe with the new one-hot encoded columns
filtered_df.reset_index(drop=True, inplace=True)
filtered_df = pd.concat([filtered_df, df_one_hot_C], axis=1)

# Drop the original 'Compulsion Type' column if you no longer need it
filtered_df.drop(columns=['Compulsion Type'], inplace=True)


In [26]:
# Using Label Encoding on Score_Category
from sklearn.preprocessing import LabelEncoder

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Apply the label encoder to the 'Score_Category' column
filtered_df['Score_Category_Encoded'] = label_encoder.fit_transform(filtered_df['Score_Category'])

In [ ]:
# Display the updated dataframe
filtered_df.head()

### Diagnostic Analytics

1. Is there a relationship between diagnosis of anxiety and the severity of symptoms?

In [ ]:
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt

# Contingency table for Anxiety Diagnosis vs Symptom Severity
contingency_table = pd.crosstab(filtered_df['Anxiety Diagnosis'], filtered_df['Score_Category'])

# Perform chi-square test
chi2, p, dof, expected = stats.chi2_contingency(contingency_table)

# Print the chi-square test result
print(f"Chi-Square Test Statistic: {chi2}, p-value: {p}")

# Visualize the relationship using a heatmap
sns.heatmap(contingency_table, annot=True, fmt='d', cmap='YlGnBu', cbar=True)
plt.title('Anxiety Diagnosis vs Symptom Severity')
plt.show()

The p-value is lower than the common significance level of p-value, 0.05, so there is a statistically significant relationship between diagnosis of anxiety and the severity of the symptoms. 

2. Is there a relationship between a family history of OCD and the severity of symptoms?

In [ ]:
# Contingency table for Family History vs Symptom Severity
contingency_table = pd.crosstab(filtered_df['Family History of OCD'], filtered_df['Score_Category'])

# Perform chi-square test
chi2, p, dof, expected = stats.chi2_contingency(contingency_table)

# Print the chi-square test result
print(f"Chi-Square Test Statistic: {chi2}, p-value: {p}")

# Visualize the relationship using a heatmap
sns.heatmap(contingency_table, annot=True, fmt='d', cmap='YlGnBu', cbar=True)
plt.title('Family History vs Symptom Severity')
plt.show()

The Chi-Square test statistic (1.23) is relatively low indicating a weaker association.  
Since the p-value is greater than 0.05, there is no statistically significant association between having a family history of OCD and the severity of symptoms (whether low or high).

3. Is there a relationship between diagnosis of depression and the severity of symptoms?

In [ ]:
# Contingency table for Depression Diagnosis vs Symptom Severity
contingency_table = pd.crosstab(filtered_df['Depression Diagnosis'], filtered_df['Score_Category'])

# Perform chi-square test
chi2, p, dof, expected = stats.chi2_contingency(contingency_table)

# Print the chi-square test result
print(f"Chi-Square Test Statistic: {chi2}, p-value: {p}")

# Visualize the relationship using a heatmap
sns.heatmap(contingency_table, annot=True, fmt='d', cmap='YlGnBu', cbar=True)
plt.title('Depression Diagnosis vs Symptom Severity')
plt.show()

The p-value is higher than the common value for significance, 0.05, so there is no statistically significant relationship between them. Thus, we cannot conclude that the presence or absence of depression in OCD is not related to the severity of the symptoms.

4. Below, we try to evaluate the corelation between Age and Total_Score.

In [ ]:
# Calculate the Pearson correlation coefficient
correlation_coefficient = filtered_df['Age'].corr(filtered_df['Total_Score'])
print(f'Pearson correlation coefficient between Age and Total_Score: {correlation_coefficient:.2f}')

# Create a scatter plot to show the relationship
fig1 = px.scatter(filtered_df, x='Age', y='Total_Score', 
                 title='Scatter Plot of Age vs Total Score',
                 labels={'Age': 'Age', 'Total_Score': 'OCD Severity (Y-BOCS Total Score)'},
                 trendline='ols')  # Add a trendline to visualize the linear relationship

# Show the figure
fig1.show()

This indicates a very weak positive correlation between them, which would be almost negligible. Age and total score can be assumed to be nearly uncorrelated.

5. What is the correlation between the duration of symptoms and OCD severity?

In [ ]:
# Calculate the Pearson correlation coefficient
correlation_coefficient = filtered_df['Duration of Symptoms (months)'].corr(filtered_df['Total_Score'])
print(f'Pearson correlation coefficient between Duration of symptoms and Total_Score: {correlation_coefficient:.2f}')

# Create a scatterplot
fig5 = px.scatter(filtered_df, x='Duration of Symptoms (months)', y='Total_Score', 
                  title='Relationship between Duration of Symptoms and OCD Severity',
                  labels={'Duration of Symptoms (months)': 'Duration of Symptoms (months)', 'Total_Score': 'OCD Severity (Y-BOCS Total Score)'}, 
                  trendline='ols')
fig5.show()

The Pearson correlation coefficient between the duration of symptoms and OCD severity(total score) is  0.03, which is an extremely weak positive correlation. Thus, it suggests that there is no relationship between the duration of symptoms and OCD severity.

6. How are Age and Duration of Symptoms related?

In [ ]:
from sklearn.linear_model import LinearRegression
import numpy as np

# Calculate the Pearson correlation coefficient
correlation_coefficient = filtered_df['Age'].corr(filtered_df['Duration of Symptoms (months)'])
print(f'Pearson correlation coefficient between Age and Duration of symptoms: {correlation_coefficient:.2f}')


# Create a scatterplot
fig6 = px.scatter(filtered_df, x='Age', y='Duration of Symptoms (months)', 
                  title='Relationship between Age and Duration of Symptoms',
                  labels={'Age': 'Age', 'Duration of Symptoms (months)': 'Duration of Symptoms (months)'}, 
                  trendline='ols')
fig6.show()

The Pearson correlation coefficient between the duration of symptoms and age is also 0.07. So, there is a weak positive association, meaning that if age increases, the duration of symptoms may increase, but it is a small relationship and can be negligible.

### Predictive Analytics: Classification

We choose to predict the severity of OCD symptoms (by predicting Score_Category). "Score_Category_Encoded" is the target class. It is binary.
Going by the literature review and subject matter expertise within the team, features that are deemed to be necessary to create a prediction model are: Age, Gender, Family History of OCD, Duration of Symptoms, Depression Diagnosis, Anxiety Diagnosis, Type of Obsession, Type of Compulsion. 

Prediction task: Build a model that predicts severity of OCD symptoms by assigning a category to the Y-BOCS scores of patients.

In [44]:
# Drop columns that are not required
new_df = filtered_df.drop(columns=['Total_Score', 'Score_Category', 'Age Group', 'Condition', 'Gender'])

new_df=new_df.astype(float)

# Convert new_df to CSV
new_df.to_csv('output_file.csv', index=False)  # index=False to avoid writing row numbers

In [ ]:
# View the concise dataframe
new_df.head()

In [ ]:
new_df.dtypes

In [ ]:
# Convert pd df into numpy array
new_df_na = new_df.values
print(new_df_na)

Checking for class imbalance

In [ ]:
# Plot histogram for target class
fig = px.histogram(new_df, x='Score_Category_Encoded', nbins=2, 
                   title='Distribution of Score Category (Encoded)',
                   labels={'Score_Category_Encoded': 'Score Category (Encoded)'},
                   text_auto=True
)

# Update the layout for better visualization
fig.update_layout(
    xaxis_title='Score Category (Encoded)',
    yaxis_title='Count',
    bargap=0.8,
    showlegend=False
)

# Show the plot
fig.show()

While there doesn't seem to be a significant class imbalance, we still choose to perform Stratified train-test partitioning to ensure that the distribution of classes in both the training and testing sets remains similar to the original dataset.

In [49]:
from sklearn.model_selection import train_test_split

# Separate features (x) and target variable (y)
x = new_df_na[:, :-1]  # Features
y = new_df_na[:, -1].astype(float)  # Target variable

import numpy as np
RANDOM_SEED = 2000
np.random.seed(RANDOM_SEED)

# Create splits from the normalized dataset.
x_train, x_test, y_train, y_test = train_test_split(x, y,
                                                    test_size=0.2,
                                                    random_state=RANDOM_SEED)


In [50]:
# Models used
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

In [ ]:
models = {
    "DT_1": DecisionTreeClassifier(max_depth=3),
    "DT_2": DecisionTreeClassifier(max_depth=5),
    "RF_1": RandomForestClassifier(n_estimators=10, max_depth=3),
    "RF_2": RandomForestClassifier(n_estimators=100, max_depth=5),
    "KNN_1": KNeighborsClassifier(n_neighbors=3),
    "KNN_2": KNeighborsClassifier(n_neighbors=7),
    "SVM_1": SVC(kernel='rbf'),
    "SVM_2": SVC(kernel='linear'),
    "SVM_3": SVC(kernel='sigmoid'),
    "SVM_4": SVC(kernel='linear', class_weight= {0:2, 1:1}),
    "LR_1": LogisticRegression(),
    "LR_2": LogisticRegression(class_weight= "balanced"),
    "LR_3": LogisticRegression(class_weight= {0:2, 1:1})
}

# Iterate over the dictionary as follows:
for name, classifier in models.items():
    print("The name of the classifier is:", name, " and it is a sklearn object:", classifier)

In [52]:
# Individual performance metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import time

# Pandas dataframe to store results of each variable on the 10 classifiers
results = pd.DataFrame({
                        "classifier_name": [],
                        "training_time": [],
                        "prediction_time": [],
                        "accuracy": [],
                        "precision": [],
                        "recall": [],
                        "f1": []
                        })

In [ ]:
# Apply the classifier in each dataset
for name, classifier in models.items():

    # Start measuring the training time
    t_start = time.time()
    # The code below applies the specific classifier from the loop on the data
    classifier.fit(x_train,y_train)
    # Stop measuring training time
    t_end = time.time()
    # Calculate Variable of interest `training_time`
    training_time = t_end - t_start
    #######

    # Start measuring the prediction time
    t_start = time.time()
    # Predict on test set
    y_predicted = classifier.predict(x_test)
    # Stop measuring prediction time
    t_end = time.time()
    # Calculate Variable of interest `prediction_time`
    prediction_time = t_end - t_start
    #######


    # Get performance metrics
    accuracy_result = accuracy_score(y_test, y_predicted)
    precision_result = precision_score(y_test, y_predicted, zero_division=0)
    recall_result = recall_score(y_test, y_predicted)
    f1_result = f1_score(y_test, y_predicted)
    #######


    #### Generate the results to populate the pandas.DataFrame
    this_result = pd.DataFrame({
                    "classifier_name": [name],
                    "training_time": [training_time],
                    "prediction_time": [prediction_time],
                    "accuracy": [accuracy_result],
                    "precision": [precision_result],
                    "recall": [recall_result],
                    "f1": [f1_result]
                    })
    # Append to the main dataframe with the results
    results = pd.concat([results, this_result], axis=0, ignore_index=True)

results

We think that for this particular classification problem, a model should be chosen based on its precision and recall.  
Precision is important because incorrectly labeling a patient as having high severity is costly (false positives). Recall is important as missing a high-severity patient is far from good (false negatives).  
As f1 score is the harmonic mean of precision and recall, it balances both, false positives and the false negatives. Hence we chose the model with the best f1 score.  
Random Forest handles non linear relationships, is resistant to over fitting and provides insights into feature importance. Therefore we choose the RF_2 model from the above list.

In [ ]:
rf_model = models['RF_2']

import joblib

# Save the rf_model to a file
joblib.dump(rf_model, 'rf_model.pkl')

### Prescriptive Analytics

SHAP

For prescriptive analytics, we used SHAP visualization method. Here we use the SHAP summary plot, we use it for multiple predictions (x_test) to provide a comprehensive overview of how each feature in all instances has an impact on the random forest model predictions in both magnitude and direction(positive or negative). However, the dashboard uses a single prediction to visualize the impact of the features of a particular instance on the model (prediction from user input). Thus, the dashboard users, especially clinicians, can understand how each feature contributes to the model prediction result (high or low severity) of the patient examined.

The plot consists of:

Vertical Axis (Feature Names): The features are listed on the y-axis, sorted by their global importance (top features are most influential on the model's predictions across the entire dataset).  

Horizontal Axis (SHAP Values): The x-axis shows the SHAP values, representing the impact of a feature on the model's output. SHAP values can be positive (increasing the predicted output) or negative (decreasing the predicted output), depending on the specific feature value.  

Each Point Represents an Instance. The position of the dot along the x-axis shows the magnitude and direction of that feature's effect on the prediction for that instance. A dot far to the right means the feature strongly pushes the prediction higher, while one far to the left means it strongly pushes the prediction lower.  

Color Gradient (Feature Value): The color of each dot represents the value of the feature for that instance (e.g., blue for low values, red for high values). If high values of a feature (red) push the prediction higher (positive SHAP values), and low values (blue) push it lower (negative SHAP values), you can infer a positive relationship between that feature and the prediction.

In [ ]:
import shap

# Explainer initialization and SHAP values computation
explainer = shap.TreeExplainer(models['RF_2'])  # Using the DT_2 model from your dictionary
shap_values = explainer.shap_values(x_test)

# Check the shape of SHAP values and x_test_df
print(f"Shape of shap_values: {shap_values.shape}")
print(f"Shape of x_test: {x_test.shape}")  # Feature matrix
print(f"Shape of shap_values: {shap_values[:,:,1].shape}")

In [ ]:
feat_names=list(new_df.columns)[:-1] #exclude last column which is the label

#Summary plot for class 1 (High)
shap.summary_plot(shap_values[:, :, 1], x_test, feature_names=feat_names)

In [ ]:
#Summary plot for class 0 (Low)
shap.summary_plot(shap_values[:, :, 0], x_test, feature_names=feat_names)

In [ ]:
#Choose index between 0 and 83 as x_test contains 84 instances
idx = 5
shap.summary_plot(shap_values[idx, :], [x_test[idx]], feature_names=feat_names)
print(f"Prediction result: {y_predicted[idx]}")